In [112]:
!pip install rank_bm25
!git clone https://github.com/cr-nlp/project1-2023.git

import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
import math
nltk.download('stopwords')
nltk.download('punkt')


def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()

	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value

	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score
	return dicDoc, dicReq, dicReqDoc


fatal: destination path 'project1-2023' already exists and is not an empty directory.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [138]:
	dicDoc, dicReq, dicReqDoc = loadNFCorpus()

In [139]:
def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops


In [140]:
def docsandreqsToKeep(startDoc, endDoc):

    dicDoc, dicReq, dicReqDoc = loadNFCorpus()

    docsToKeep = []
    reqsToKeep = []
    dicReqDocToKeep = defaultdict(dict)

    i = startDoc
    for reqId in dicReqDoc:
        if i > (endDoc - startDoc):  # nbDocsToKeep:
            break
        for docId in dicReqDoc[reqId]:
            dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
            docsToKeep.append(docId)
            i = i + 1
        reqsToKeep.append(reqId)
    docsToKeep = list(set(docsToKeep))

    return docsToKeep, reqsToKeep, dicReqDocToKeep

In [141]:
docsToKeep, reqsToKeep, dicReqDocToKeep = docsandreqsToKeep(0,150)

In [142]:
from rank_bm25 import BM25Okapi
def Tokenize(docsToKeep, reqsToKeep,dicDoc,dicReq):
    corpusDocTokenList = []
    corpusReqTokenList = {}
    corpusDicoDocName={}
    i = 0
    for k in docsToKeep:
      docTokenList = text2TokenList(dicDoc[k])
      corpusDocTokenList.append(docTokenList)
      corpusDicoDocName[k] = i
      i = i + 1

    #print("reqs...")
    corpusReqName=[]
    corpusDicoReqName={}
    i = 0
    for k in reqsToKeep:
      reqTokenList = text2TokenList(dicReq[k])
      corpusReqTokenList[k] = reqTokenList
      corpusReqName.append(k)
      corpusDicoReqName[k] = i
      i = i + 1
    return corpusDocTokenList,corpusDicoDocName,corpusReqTokenList,corpusReqName,corpusDicoReqName
    #print("bm25 doc indexing...")


In [143]:
corpusDocTokenList,corpusDicoDocName,corpusReqTokenList,corpusReqName,corpusDicoReqName = Tokenize(docsToKeep, reqsToKeep,dicDoc,dicReq)

In [144]:
def init_vb(corpusDocTokenList):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        doc_len=[]
        doc_freqs=[]
        corpus_size=0
        avgdl=0
        for document in corpusDocTokenList:
            doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

            corpus_size += 1

        avgdl = num_doc / corpus_size
        return num_doc,doc_len,doc_freqs,corpus_size,avgdl,nd

def calc_idf(nd,corpus_size,epsilon=0.25):
    """
    Calculates frequencies of terms in documents and in corpus.
    This algorithm sets a floor on the idf values to eps * average_idf
    """
    # collect idf sum to calculate an average idf for epsilon value
    idf_sum = 0
    dicidf={}
    # collect words with negative idf to set them a special epsilon value.
    # idf can be negative if word is contained in more than half of documents
    negative_idfs = []
    for word, freq in nd.items():
        idf = math.log(corpus_size - freq + 0.5) - math.log(freq + 0.5)
        dicidf[word] = idf
        idf_sum += idf
        if idf < 0:
           negative_idfs.append(word)
    average_idf = idf_sum / len(dicidf)
    eps = epsilon * average_idf
    for word in negative_idfs:
        dicidf[word] = eps
    return dicidf

def Get_score(corpusDocTokenList,query,k1=1.5,b=0.75):

  num_doc,doc_len,doc_freqs,corpus_size,avgdl,nd = init_vb(corpusDocTokenList)

  score = np.zeros(len(corpusDocTokenList))
  dicidf = calc_idf(nd,corpus_size)
  doc_len = np.array(doc_len)


  for q in query:
      q_freq = np.array([(doc.get(q) or 0) for doc in doc_freqs])
      score += (dicidf.get(q) or 0) * (q_freq * (k1 + 1) /(q_freq + k1 * (1 - b + b * doc_len / avgdl)))
  return score

In [145]:
  num_doc,doc_len,doc_freqs,corpus_size,avgdl,nd = init_vb(corpusDocTokenList)


In [146]:
  from sklearn.metrics import ndcg_score

  def Scoring(corpusDocTokenList,corpusDicoDocName,corpusReqTokenList,corpusReqName,corpusDicoReqName):

    #bm25 = BM25Okapi(corpusDocTokenList)
    #bm25 = Get_score(corpusDocTokenList)#,reqTokenList)

    ndcgCumul=0
    corpusReqVec={}
    ndcgBM25Cumul=0
    nbReq=0
    ndcgTop=5

    for req in corpusReqTokenList:

      reqTokenList = corpusReqTokenList[req]
      #doc_scores = bm25.get_scores(reqTokenList)
      doc_scores = Get_score_2(corpusDocTokenList,reqTokenList)
      trueDocs = np.zeros(len(corpusDocTokenList))

      for docId in corpusDicoDocName:
        if req in dicReqDocToKeep:
          if docId in dicReqDocToKeep[req]:
            #get position docId
            posDocId = corpusDicoDocName[docId]
            trueDocs[posDocId] = dicReqDocToKeep[req][docId]
            #print("TOKEEP=",docId)
            #print(trueDocs)
      ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
      nbReq = nbReq + 1
    ndcgBM25Cumul = ndcgBM25Cumul / nbReq
    print("ndcg bm25=",ndcgBM25Cumul)
    return ndcgBM25Cumul

In [147]:
Scoring(corpusDocTokenList,corpusDicoDocName,corpusReqTokenList,corpusReqName,corpusDicoReqName)

ndcg bm25= 0.20395955831078683


0.20395955831078683

In [57]:
 for req in corpusReqTokenList:
  reqTokenList = corpusReqTokenList[req]

  score= Get_score(corpusDocTokenList,reqTokenList)

In [126]:
from gensim.models import Word2Vec

import gensim.downloader

model = gensim.downloader.load('glove-wiki-gigaword-200')

#model = Word2Vec(sentences=traintokencorpus, vector_size=100, window=5, min_count=1, workers=4)


[==================================================] 100.0% 252.1/252.1MB downloaded


In [153]:
from sklearn.metrics.pairwise import cosine_similarity

def Get_score_2(corpusDocTokenList,query):

  query_word2vec = np.array([model[word] for word in query if word in model])


  cosine_sim_word2vec = np.array([np.mean([cosine_similarity(query_word2vec[wordo].reshape(1,-1), model[word].reshape(1, -1)).flatten()[0] for word in corpusDocTokenList[a] for wordo in range(len(query_word2vec)) if word in model],axis=0)\
                        for a in range(len(corpusDocTokenList))])
  return cosine_sim_word2vec


In [148]:
query_word2vec

[array([-0.04545643,  0.02769067,  0.56294966], dtype=float32),
 array([-0.07642844,  0.04368665,  0.5795835 ], dtype=float32),
 array([-0.04606264,  0.04265467,  0.51916176], dtype=float32),
 array([-0.06154141,  0.02712312,  0.4563036 ], dtype=float32)]

In [154]:
Get_score_2(corpusDocTokenList,corpusReqTokenList['PLAIN-1'])

KeyboardInterrupt: ignored

In [155]:
query_word2vec

[array([-0.04545643,  0.02769067,  0.56294966], dtype=float32),
 array([-0.07642844,  0.04368665,  0.5795835 ], dtype=float32),
 array([-0.04606264,  0.04265467,  0.51916176], dtype=float32),
 array([-0.06154141,  0.02712312,  0.4563036 ], dtype=float32)]

In [91]:
Get_score(corpusDocTokenList,corpusReqTokenList['PLAIN-1'])

array([315.76514397,  55.49115531,  85.8978511 , 236.30320682,
       128.98258025, 182.88338228, 158.17910857, 151.16683625,
       141.42214796,  64.94309814,  60.7362493 ,  79.27589656,
       162.45939749,  73.0887188 , 135.8151554 ,  38.24943811,
       126.54801046, 259.53255952, 140.65161621, 527.38803394,
       173.89832714,  78.93975084,  76.23746938, 309.11287953,
       883.9652338 , 218.66399796,  77.96489064,  91.38049014,
       141.93189619, 134.53659709, 346.0772379 , 110.31245056,
        96.13223987, 169.23244499, 167.54699424, 212.92510528,
        52.04778373,  93.36418667,  80.50929588, 143.65834223,
       325.27245992,  67.46323317, 115.27236995, 319.36550771,
       133.43666882, 126.41327688, 255.62933675, 152.30171819,
       205.71112885, 199.89612992,   2.12717939,  44.5224862 ,
        88.09727202,  88.44759774, 171.48094194, 142.08648354,
       104.19696708, 340.62076947,  79.24539543,  12.70927888,
        61.43545935,  47.17334829, 583.61699782, 134.08

In [38]:
query_word2vec = [np.mean([model.wv[word] for word in corpusReqTokenList[query]if word in model.wv],axis=0) for query in corpusReqTokenList.keys() ]
query_word2vec

[array([-0.04545643,  0.02769067,  0.56294966], dtype=float32),
 array([-0.07642844,  0.04368665,  0.5795835 ], dtype=float32),
 array([-0.04606264,  0.04265467,  0.51916176], dtype=float32),
 array([-0.06154141,  0.02712312,  0.4563036 ], dtype=float32)]

In [40]:
np.array(query_word2vec).reshape(1, -1)

array([[-0.04545643,  0.02769067,  0.56294966, -0.07642844,  0.04368665,
         0.5795835 , -0.04606264,  0.04265467,  0.51916176, -0.06154141,
         0.02712312,  0.4563036 ]], dtype=float32)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

tokenized_documents = [doc.split() for doc in documents]
word2vec_model = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4)

query_tfidf = tfidf_vectorizer.transform([query])
query_word2vec = np.mean([word2vec_model.wv[word] for word in query.split() if word in word2vec_model.wv], axis=0)
